# Wave Download Speed Test

### Description:
This script porpuse is to execute a speed test of downloading wave to a Proteus device.
The script creates a sine waves in size of 1 Gigabyte memory and downloads it to the device and measure the time it takes.


Proteus release - Aurora

Note: For accurate time readings DLL connection is preferred with tabor proteus services turned off, No other connection required to run this script.

In [1]:
slot_id = '2'
max = 0
channb = 1 #input("Channel number to test the waveform speed")
test_success = True

In [2]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('..//SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

# Connect to instrument
#inst = connect('192.90.70.22')
inst = connect("2",Auto=False)
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))
# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':TRACe:DELete:ALL')
# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))
## Configure Generator

segnum = 1
seglen = 268304384 #102400000
num_cycles=2000
cyclelen = seglen / num_cycles
# Build waveform
x = np.linspace(start=0, stop=2 * np.pi * num_cycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
if dac_mode == 16:
    wave = y.astype(np.uint16)
else:
    wave = y.astype(np.uint8)

# data-type code: 'B' = uint8, 'H' = uint16
data_type = 'H' if dac_mode == 16 else 'B'

# for loop iterating over the range of sampling rate from 1GHz to 9Ghz with a step of .5GHz

for in_freq in np.arange(1000,9500,500):
    inst.send_scpi_cmd(":FREQ:RAST {0}MHz".format(in_freq))
    resp = inst.send_scpi_query(":FREQ:RAST?")
    freq = float(resp)/1e6
    #print ("Press enter to download the waveform for sampling rate :{0}MHz".format(freq))
    #input()
    inst.timeout = 30000
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)

    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    inst.send_scpi_cmd(cmd)

    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    inst.timeout = 30000

    #start_time = time.time()
    tic = time.perf_counter()

    inst.write_binary_data(':TRAC:DATA', wave)

    #end_time = time.time()
    #time_lapsed = end_time - start_time

    #print('Downloaded the wave in "{0}" seconds'.format(time_lapsed))

    toc = time.perf_counter()
    timer = float(toc-tic)
    if timer <10 :
        print('Sampling rate = {0},Downloaded the wave in "{1}" seconds'.format(in_freq,timer))
    else:
        test_success = False
        print("Test fail ")


    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
         print('ERROR: "{0}" after writing binary values'.format(resp))

    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

if timer> max:
    max = timer
    freq = in_freq
print("Maximum time to download waveform is {0} at frequency {1}MHz".format(max,freq))


PXI connect
Load TEProteus.dll from : C:\Windows\system32\TEProteus.dll
Trying to connect to PXI-slot:2
Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2,
Model: P9484M 
DAC 16 bits
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes
Sampling rate = 1000,Downloaded the wave in "0.7537246000001687" seconds
Sampling rate = 1500,Downloaded the wave in "0.7410679000004166" seconds
Sampling rate = 2000,Downloaded the wave in "0.7739711999993233" seconds
Sampling rate = 2500,Downloaded the wave in "0.7346059999999852" seconds
Sampling rate = 3000,Downloaded the wave in "7.626352699999188" seconds
Sampling rate = 3500,Downloaded the wave in "7.639873099999932" seconds
Sampling rate = 4000,Downloaded the wave in "7.6718857999994725" seconds
Sampling rate = 4500,Downloaded the wave in "7.617130300000099" seconds
Sampling rate = 5000,Downloaded the wave in "7.70704139999998" seconds
Sampling rate = 5500,Downloaded the wave in "9.097800800000186" seconds
Sampling 

In [3]:
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity_after_download = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity_after_download))
disconnect()  
if test_success:
    print('Test successed')
else:
    print('Test failed')

Available memory per DDR: 16,903,176,192 wave-bytes
Test successed
